In [15]:
%load_ext autoreload
%autoreload 2

In [4]:
class EffectType_s():
    def __init__(self, tgt, effect):
        self.tgt = tgt
        self.effect = effect
    def __repr__(self):
        return f"ET({self.tgt},{self.effect})"
#     def __eq__(self, other):
# 		return z3.eq(self.pvar, other.pvar) and self.index == other.index
# 	def __lt__(self, other):
# 		# TODO incorporate pvar type into sort.
# 		if str(self.pvar) > str(other.pvar): return False
# 		if str(self.pvar) == str(other.pvar) and self.index >= other.index: return False
# 		return True
# 	def __hash__(self):
# 		return hash((hash(self.pvar), hash(self.index)))

In [1]:
from action import Action
from PDDL import PDDL_Parser
import sys, pprint
from collections import OrderedDict
import re, copy
import itertools
import z3
from skill_classes import EffectTypePDDL

**TODO**

* Construct pvars
* Create Skill with parameters
    * Create EffectType with str index, instead of int index. This way we can put the details of the effect into the index. Maybe add parameters attribtue to EffectType?

* Come up with better name for 'params' - the pvars used to calculate effects. Ex. for the effect 'person.y += person.leg_length', person.leg_length is a param. Params are treated like pvars in precondition, except they cannot be causally linked

In [2]:
def list_is_flat(l):
    for x in l: 
        if isinstance(x, list):
            return False
    return True

In [18]:
def make_EffectType_s(l, is_add = True):
    if list_is_flat(l):
        tgt = l
        effect = is_add
    else:
        tgt = l[1]
        effect = [l[0],l[-1]]
    return EffectType_s(tgt, effect)

In [19]:
def action2EffectTypes(act):
    return [make_EffectType_s(x,True) for x in act.add_effects] + [make_EffectType_s(x,False) for x in act.del_effects]

In [3]:
zeno_dom = "examples/zeno/zeno.pddl"
zeno_prob = "examples/zeno/pb1.pddl"

In [21]:
dinner_dom = "./examples/dinner/dinner.pddl"
dinner_prob = "./examples/dinner/pb1.pddl"

In [4]:
# domain, problem = dinner_dom, dinner_prob
domain, problem = zeno_dom, zeno_prob

parser = PDDL_Parser()
print('----------------------------')
# pprint.pprint(parser.scan_tokens(problem))
print('----------------------------')
parser.parse_domain(domain)
parser.parse_problem(problem)
print('Domain name: ' + parser.domain_name)
print(f'~~~Types~~~')
for t in parser.types: print(t)
print(f'~~~Type Hierarchy~~~')
for k,v in parser.type_hierarchy.items():
    print(f"{k}: {v}")
print(f'~~~Predicates~~~')
for nm, args in parser.predicates.items():
    print(f"{nm}({args}")
#     print("")
print('----------------------------')
print(f'~~~Functions~~~')
for nm, args in parser.functions.items():
    print(f"{nm}({args}")
#     print("")
print('----------------------------')
print('~~~Actions~~~')
for act in parser.actions:
    print(act)
    # print("effect types:")
    # for ef in action2EffectTypes(act): print(f"\t{ef}")
    print("\n")
print('----------------------------')
print('Problem name: ' + parser.problem_name)
print('Objects: ' + str(parser.objects))
print('State: ' + str(parser.state))
print('Positive goals: ' + str(parser.positive_goals))
print('Negative goals: ' + str(parser.negative_goals))

----------------------------
----------------------------
Domain name: zenotravel
~~~Types~~~
aircraft
city
locatable
object
person
~~~Type Hierarchy~~~
aircraft: []
city: []
locatable: ['aircraft', 'person']
object: ['city', 'locatable']
person: []
~~~Predicates~~~
located(OrderedDict([('?x', 'locatable'), ('?c', 'city')])
in(OrderedDict([('?p', 'person'), ('?a', 'aircraft')])
----------------------------
~~~Functions~~~
fuel(OrderedDict([('?a', 'aircraft')])
distance(OrderedDict([('?c1', 'city'), ('?c2', 'city')])
slow-burn(OrderedDict([('?a', 'aircraft')])
fast-burn(OrderedDict([('?a', 'aircraft')])
capacity(OrderedDict([('?a', 'aircraft')])
total-fuel-used(OrderedDict()
onboard(OrderedDict([('?a', 'aircraft')])
zoom-limit(OrderedDict([('?a', 'aircraft')])
----------------------------
~~~Actions~~~
action: board
  parameters: [['?p', 'person'], ['?a', 'aircraft'], ['?c', 'city']]
  positive_preconditions: [['located', '?p', '?c'], ['located', '?a', '?c']]
  negative_preconditions: [

In [23]:
parser.types

['aircraft', 'city', 'locatable', 'object', 'person']

In [26]:
nm, var_nm_types = list(parser.predicates.items())[0]

print(nm)
print(var_nm_types)

# IMPORTANT: This creates pvars that lose all notion of subtypes.
# For instance, we now have a bunch of pvars for 
# located(locatable, city) and if we're asked for subsets of these vars
# that are located(person, city), we can't get them!!!
varnm2objects = OrderedDict()
for varnm, vartype in var_nm_types.items():
    subtypes = parser.get_subtypes([vartype])
    valid_objects = []
    for st in subtypes:
        if st in parser.objects.keys():
            valid_objects.extend(parser.objects[st])
    varnm2objects[varnm] = valid_objects

print(f"varnm2objects:\n{varnm2objects}\n")

# Note: the * operator here is used to unpack the values so the itertools.product function
# can operate over all of them as opposed to producting a single list (which is useless)
grounded_pvar_predicate_list = list(itertools.product(*varnm2objects.values()))
print(grounded_pvar_predicate_list)

predicate_z3_list = []
for arg_tuple in grounded_pvar_predicate_list:
    predicate_z3_list.append(z3.Bool(nm+str(arg_tuple)))
print(predicate_z3_list)

located
OrderedDict([('?x', 'locatable'), ('?c', 'city')])
varnm2objects:
OrderedDict([('?x', ['plane1', 'person1', 'person2', 'person3']), ('?c', ['city0', 'city1', 'city2'])])

[('plane1', 'city0'), ('plane1', 'city1'), ('plane1', 'city2'), ('person1', 'city0'), ('person1', 'city1'), ('person1', 'city2'), ('person2', 'city0'), ('person2', 'city1'), ('person2', 'city2'), ('person3', 'city0'), ('person3', 'city1'), ('person3', 'city2')]
[located('plane1', 'city0'), located('plane1', 'city1'), located('plane1', 'city2'), located('person1', 'city0'), located('person1', 'city1'), located('person1', 'city2'), located('person2', 'city0'), located('person2', 'city1'), located('person2', 'city2'), located('person3', 'city0'), located('person3', 'city1'), located('person3', 'city2')]


In [25]:
a = parser.actions[0]; print(a)

action: board
  parameters: [['?p', 'person'], ['?a', 'aircraft'], ['?c', 'city']]
  positive_preconditions: [['located', '?p', '?c'], ['located', '?a', '?c']]
  negative_preconditions: []
  add_effects: [['in', '?p', '?a'], ['increase', ['onboard', '?a'], '1']]
  del_effects: [['located', '?p', '?c']]



In [13]:
a.positive_preconditions

[['located', '?p', '?c'], ['located', '?a', '?c']]

# Ground action

In [34]:
for a in parser.get_action_groundings(parser.actions[2]):
    print(a)

action: fly-slow
  parameters: [['plane1', 'aircraft'], ['city0', 'city'], ['city0', 'city']]
  positive_preconditions: [['located', 'plane1', 'city0'], ['>=', ['fuel', 'plane1'], ['*', ['distance', 'city0', 'city0'], ['slow-burn', 'plane1']]]]
  negative_preconditions: []
  add_effects: [['located', 'plane1', 'city0'], ['increase', ['total-fuel-used'], ['*', ['distance', 'city0', 'city0'], ['slow-burn', 'plane1']]], ['decrease', ['fuel', 'plane1'], ['*', ['distance', 'city0', 'city0'], ['slow-burn', 'plane1']]]]
  del_effects: [['located', 'plane1', 'city0']]

action: fly-slow
  parameters: [['plane1', 'aircraft'], ['city0', 'city'], ['city1', 'city']]
  positive_preconditions: [['located', 'plane1', 'city0'], ['>=', ['fuel', 'plane1'], ['*', ['distance', 'city0', 'city1'], ['slow-burn', 'plane1']]]]
  negative_preconditions: []
  add_effects: [['located', 'plane1', 'city1'], ['increase', ['total-fuel-used'], ['*', ['distance', 'city0', 'city1'], ['slow-burn', 'plane1']]], ['decrease'

In [36]:
def make_z3Bool_from_list(expr_list):
    return_str = precond_list[0] + '('
    for obj in precond_list[1:]:
        return_str += obj + ', '
    return_str = return_str[:-2]
    return_str += ')'
    return_expr = z3.Bool(return_str)
    return return_expr

def make_pvar_from_precond_list(precond_list):
    if(precond_list[0] not in ['>', '<', '=', '>=', '<=']):
        return make_z3Bool_from_list(expr_list)
    else:
        # TODO: how to make a z3 condition for comparison operators???
        # I'm guessing I need to use z3.eq, etc.???
        return_str = 'some_expr_involving_int_vars()'

    return return_expr

print(make_pvar_str_from_precond_list(['located', 'plane1', 'city0']))

located(plane1, city0)


In [ ]:

def make_effectType_from_effects_list(effects_list, add_or_del):
    if not(add_or_del):
        # Pretty sure this is right, but a bit unsure if a del effect implies logical inversion or if 
        # it simply assigns the specific bool to False
        z3_expr = z3.Not(make_z3Bool_from_list(effects_list))
        # Not sure if i should z3Not the z3_expr or just use the effect type to denote inversion
        return EffectTypePDDL(z3_expr, "false")

    if effects_list[0] not in ['increase', 'decrease', 'assign']:
        # This will return a z3Bool
        z3_expr = make_z3Bool_from_list(effects_list)
        return EffectTypePDDL(z3_expr, "true")

    else:
        # TODO!!!
        pass


In [ ]:
for curr_action in parser.actions:
    for grounded_action in parser.get_action_groundings(curr_action):
        precond_list = []
        for pos_precond in grounded_action.positive_preconditions:
            precond_list.append(make_pvar_from_precond_list(pos_precond))
        for neg_precond in grounded_action.negative_preconditions:
            precond_list.append(z3.Not(make_pvar_from_precond_list(neg_precond)))
        